# Module 5: Assignment 2
------------------------

Run-length encoding (RLE) is a form of lossless data compression in which runs of data (sequences in which the same data value occurs in many consecutive data elements) are stored as a single data value and count, rather than as the original run. For example,

```
Original: AAAAAAAAAAAAABBBCCD (19 characters), 
Encoded:  13A3B2C1D (7 characters)
```


This is most efficient on data that contains many such runs, for example, simple graphic images such as icons, line drawings, Conway's Game of Life, and animations. For files that do not have many runs, RLE could increase the file size.

```
Original: ABCABCABC(9 characters), 
Encoded:  A1B1C1A1B1C1A1B1C1 (18 characters)
```


RLE may also be used to refer to an early graphics file format supported by CompuServe for compressing black and white images, but was widely supplanted by their later Graphics Interchange Format (GIF). Check out this great throwback article, [Smile You're on RLE](http://csbruce.com/cbm/transactor/pdfs/trans_v7_i06.pdf).

### Lossless Compression in DNA Sequences

Lossless compression algorithms have been used for efficiently storing DNA seqeunces, but the tried-and-true `.fasta` text file has never been usurped as the standard. Take a look at some of the following approaches:
* [A Compression Algorithm for DNA Sequences and Its Applications in Genome Comparison - PubMed](https://pubmed.ncbi.nlm.nih.gov/11072342/)
* [A lossless compression algorithm for DNA sequences - PubMed](https://pubmed.ncbi.nlm.nih.gov/19887334/)
* [Biological sequence compression algorithms - PubMed](https://pubmed.ncbi.nlm.nih.gov/11700586/)



Develop a lossless compression algorithm based on RLE for storing DNA sequences. You may choose to implement an existing algorithm, modifym or create your own. Note that there are some properties of DNA, such as long repeats, that can be exploited to gain greater compression over single letter runs.

The only criteria is that your approach must be able to fully decode as the original sequence and header AND it must never be larger size than the original. Your program should be able to reade/write the encoded and decoded sequence to a file.

Test your approach using Human Chromosome 21 sequence and the Sars-CoV-2 genome. 


Calculate the compression ratio of your approach for each sequence.

$ Compression\ ratio  = (Size_{before})/(Size_{after})$

want to get similar, opposite, mirrored, reverse mirrored. want to still encode mostly similar seq. 
We can define different regions. 1 of sequences that are to be used in similar, opposite, mirrored, reversed sequences.
Another of how each sequence is modified for each each relavant copy. 
3rd is just stuff that does not fit neatly into sequences. 
We can encode these by a series of "paragraphs" separated by lines.
We can find sequences to use with getting a list of weighted differences between two sequences or with dynamic prog.
The cost for making a sequence to be copied is 3 for an extra new line and 2 for to denote this is to be copied.
The cost for making a copy is 3. For an extra new line and 2 to denote is a copy. Errors will cost 3 + log256(len) as we need to 
tell how to change, where to, and then make a new line.
10101x10101-> form x\n\n form 11\n\n
form's eight bits are xxxxyyzz. yy is a code for what type of sequence. zz is a code for how many bits of the first character are to be used.
With how things are set up, the 

We can use a modified lz where each modification has three numbers: number of indexes back to start, number of indexes to copy,
and how the copy is performed.  

In [14]:
# Write your code as a module that can be loaded in to this notebook. Load it and run it here.
import binascii
import numpy as np
entries = [("c", "00"), ("g", "01"), ("t", "10"), ("a", "11")]
opposites = {'G':'C','A':'T','C':'G','T':'A'}
acids = ['G','C','A','T']

In [15]:
def stringtodna(string):
    output = ""
    for char in string:
        #print(char)
        code = ord(char)
        binary = str(bin(code))[2:]
        while (len(binary) < 8):
            binary = '0' + binary
        for i in range(4):
            binarychunk = binary[2 * i: 2 * i + 2]
            for j in range(len(entries)):
                if (entries[j][1] == binarychunk):
                    #print("hit")
                    output += entries[j][0]                
    print(output)
    return output

def dnatostring(dnastring): 
    output = ""
    binarychar = ""
    count = 0
    for char in dnastring:
        loc = -1
        for i in range(len(entries)):
            if (entries[i][0] == char):
                loc = i
        binarychar += entries[loc][1]
        count += 1 
        if (count == 4):
            binarychar = int(binarychar, 2)
            input_array = binarychar.to_bytes(1, "big")
            ASCII_value = input_array.decode()
            output += ASCII_value
            binarychar = ""
            count = 0
    print(output)
    
def dnaloctostring(dnastringloc):
    f = open(dnastringloc, 'r')
    char = f.read(1).lower()
    count = 0
    binarychar = ""
    while (char):
        count += 1
        loc = -1
        for i in range(len(entries)):
            if (entries[i][0] == char):
                loc = i
        binarychar += entries[loc][1]
        if (loc == 4):
            input_array = binarychar.to_bytes(1, "big")
            ASCII_value = input_array.decode()
            yield ASCII_value
            binarychar = ""
        char = f.read(1).lower()
    f.close()
    
def dnatoopposite(dnastring):
    reverse = ""
    for char in dnastring:
        reverse = reverse + opposites[char]
    return reverse

In [44]:
dna = "TGGCAATGAGAAATGCG"

In [1]:
dna = "GTACGACGGAGTGTTATAAGATGGGAAATCGGATACCAGATGAAATTGTGGATCAGGTGCAAAAGTCGGCAGATATCGTTGAAGTCATAGGTGATTATGTTCAATTAAAGAAGCAAGGCCGAAACTACTTTGGACTCTGTCCTTTTCATGGAGAAAGCACACCTTCGTTTTCCGTATCGCCCGACAAACAGATTTTTCATTGCTTTGGCTGCGGAGCGGGCGGCAATGTTTTCTCTTTTTTAAGGCAGATGGAAGGCTATTCTTTTGCCGAGTCGGTTTCTCACCTTGCTGACAAATACCAAATTGATTTTCCAGATGATATAACAGTCCATTCCGGAGCCCGGCCAGAGTCTTCTGGAGAACAAAAAATGGCTGAGGCACATGAGCTCCTGAAGAAATTTTACCATCATTTGTTAATAAATACAAAAGAAGGTCAAGAGGCACTGGATTATCTGCTTTCTAGGGGCTTTACGAAAGAGCTGATTAATGAATTTCAGATTGGCTATGCTCTTGATTCTTGGGACTTTATCACGAAATTCCTTGTAAAGAGGGGATTTAGTGAGGCGCAAATGGAAAAAGCGGGTCTCCTGATCAGACGCGAAGACGGAAGCGGATATTTCGACCGCTTCAGAAACCGTGTCATGTTTCCGATCCATGATCATCACGGGGCTGTTGTTGCTTTCTCAGGCAGGGCTCTTGG"

In [78]:
#length, loc = findcopy(dna, 10)
length, loc = findreverse(dna, 10) #need +1 for reverse as last index not included
#length, loc = findboth(dna, 10)
print(length)
print(loc)
print(dna[3]) 
print(dna[10:13])

2
5
C
AAA


In [30]:
nucleic_codes = list("ACGTURYKMSWBDHVN-")
amino_codes = list("ABCDEFGHIKLMNPQRSTUVWYZX*-")
digits = list("0123456789")

def test_fasta(filename, isnucleic = False):
    f = open(filename, 'r')
    #text = f.read()
    line = f.readline().strip() #https://www.guru99.com/python-file-readline.html
    while line:
        if line[0] == '>':
            if line[1] == ' ':
                return False
            #adjust based on type
        else:
            #print(line)
            if isnucleic:
                for char in line:
                    if char not in digits and char.upper() not in nucleic_codes:
                        print(f"Error at: {line}")
                        return False
            else:
                for char in line:
                    if char not in digits and char.upper() not in amino_codes:
                        print(f"Error at: {line}")
                        return False
        line = f.readline().strip()
    f.close()
    return True

def parse_fasta(filename):
    if (not test_fasta(filename, False)):
        print("Invalid file")
    f = open(filename, 'r')
    line = f.readline().strip() 
    desc = ""
    seq = ""
    while line:
        if line[0] == '>':
            if desc != "":
                #yield (desc, seq)
                seq = ""
            desc = line[1:]
        else:
            seq += line
        line = f.readline().strip()
    # (desc, seq)
    f.close()
    return seq

In [17]:
def findcopy(dna, loc):
    highestlen = 0
    highestloc = 0
    currloc = 0
    startloc = 0
    while (startloc < loc):
        currloc = startloc
        currstart = startloc
        currend = loc
        currlen = 0
        while (currend < len(dna) and dna[currstart] == dna[currend]):
            currlen += 1
            currstart += 1
            currend += 1
        if (currlen > highestlen):
            highestlen = currlen
            highestloc = currloc
        startloc += currlen + 1
    #print(len(dna))
    return highestlen, highestloc

In [18]:
def findopposite(dna, loc):
    highestlen = 0
    highestloc = 0
    currloc = 0
    startloc = 0
    while (startloc < loc):
        currloc = startloc
        currstart = startloc
        currend = loc
        currlen = 0
        while (currend < len(dna) and opposites[dna[currstart]] == dna[currend]):
            currlen += 1
            currstart += 1
            currend += 1
        if (currlen > highestlen):
            highestlen = currlen
            highestloc = currloc
        startloc += currlen + 1
    #print(len(dna))
    return highestlen, highestloc

In [19]:
def findreverse(dna, loc):
    highestlen = 0
    highestloc = loc
    currloc = 0
    startloc = loc - 1
    while (startloc > -1):
        currloc = startloc
        currstart = startloc
        currend = loc
        currlen = 0
        while (currend < len(dna) and currstart > -1 and dna[currstart] == dna[currend]):
            currlen += 1
            currstart += -1
            currend += 1
        if (currlen > highestlen):
            highestlen = currlen
            highestloc = currloc
        startloc -= currlen + 1
    #print(len(dna))
    return highestlen, highestloc

In [20]:
def findboth(dna, loc):
    highestlen = 0
    highestloc = loc
    currloc = 0
    startloc = loc - 1
    while (startloc > -1):
        currloc = startloc
        currstart = startloc
        currend = loc
        currlen = 0
        while (currend < len(dna) and currstart > -1 and opposites[dna[currstart]] == dna[currend]):
            currlen += 1
            currstart += -1
            currend += 1
        if (currlen > highestlen):
            highestlen = currlen
            highestloc = currloc
        startloc -= currlen + 1
    #print(len(dna))
    return highestlen, highestloc

In [21]:
def encode(dna):
    output = ""
    loc = 0
    while(loc < len(dna)):
        lengths = [0,0,0,0]
        locs = [0,0,0,0]
        a, b = findcopy(dna, loc)
        lengths[0] = a
        locs[0] = b
        a, b = findreverse(dna, loc)
        lengths[1] = a
        locs[1] = b
        a, b = findopposite(dna, loc)
        lengths[2] = a
        locs[2] = b
        a, b = findboth(dna, loc)
        lengths[3] = a
        locs[3] = b
        #copy = 0
        #reverse = 0
        #opposite = 0
        #both = 0
        
        length = max(lengths) 
        if (length > 3): #ensure this is actually worth incrementing
            index = lengths.index(length)
            output += str(locs[index])
            output += str(length)
            output += str(index)
            loc += length
        else:
            output += dna[loc]
            loc += 1
    return output     

In [69]:
#dna = "TGGCAATGAGAAATGCG"
encoded = encode(dna)
print(encoded)

TGGCAATGAGA440CG


In [22]:
def decode(encoded):
    output = ""
    encodedloc = 0
    #outputloc = 0
    while (encodedloc < len(encoded)):
        if (encoded[encodedloc] in acids):
            output += encoded[encodedloc] 
            encodedloc += 1
            #outputloc += 1
        else:
            loc = int(encoded[encodedloc])
            encodedloc += 1
            length = int(encoded[encodedloc])
            encodedloc += 1
            method = int(encoded[encodedloc])
            encodedloc += 1
            for i in range(length):
                if (method == 0):
                    output += output[loc]
                    loc += 1
                elif (method == 1):
                    output += opposites[output[loc]]
                    loc += 1
                elif (method == 2):
                    output += output[loc]
                    loc -= 1
                else:
                    output += opposites[output[loc]]
                    loc -= 1
    return output
        

In [83]:
print(decode(encoded))

TGGCAATGAGAAATGCG


In [32]:
path = "C:/Users/ewu15/jup/Scientific Computing/week 1/data/data/sars-cov2-genome-fasta-filtered.txt"
dna = parse_fasta(path)
#print(dna)

In [33]:
encoded = encode(dna)

In [34]:
print(encoded)

...


In [37]:
f = open("C:/Users/ewu15/jup/Scientific Computing/week 1/data/data/cov-genome-comp.txt", 'w')
f.write(encoded)
f.close()

30.3kB is the original size. 24.2kB is the compressed size. The ratio is 1.25.